# Step 1

- Optimizacion Bayesiana de hiperparametros de  rpart
- 10-repeated 5-fold Cross Validation
- trabaja con la clase_ternaria (más lento -puede tardar un día-, pero certero que la binaria)

In [1]:
# Limpiamos el entorno
rm(list = ls())
gc(verbose = FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,627610,33.6,1404617,75.1,994503,53.2
Vcells,1149700,8.8,8388608,64.0,1769393,13.5


In [2]:
# Librerías necesarias
require("data.table")
require("rpart")
require("DiceKriging")
require("mlrMBO")
require("rlist")
require("parallel")

Loading required package: data.table

Loading required package: rpart

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Loading required package: checkmate


Attaching package: 'checkmate'


The following object is masked from 'package:DiceKriging':

    checkNames


Loading required package: rlist

Loading required package: parallel



In [3]:
# Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:/Users/German/Desktop/dmeyf2023/German/Clase 3") # Establezco el Working Directory
#"C:/Users/German/Documents/dmeyf2023/German/Clase 3" #Laptop

In [4]:
# Defino la  Optimizacion Bayesiana
PARAM <- list()

PARAM$home <- "C:/Users/German/Desktop/dmeyf2023/German/Clase 3"

PARAM$experimento <- "HT3220"

# Aqui van las 10 semillas que hacen el 10-repeated
#  si se pone una sola semilla, se esta haciendo solo 5-fold xval
PARAM$semilla_azar <- c(279511, 279523, 279541, 279551, 279571)

In [5]:
# Cargamos el dataset
#dataset <- fread("competencia_01.csv")

# Nos quedamos solo con el 202103
#dataset <- dataset[foto_mes == 202103]

In [6]:
# folds del cross validation
PARAM$xval_folds <- 5

# cantidad de iteraciones de la Optimizacion Bayesiana
PARAM$BO_iter <- 300

# la letra L al final de 1L significa ENTERO
PARAM$hs <- makeParamSet(
  makeNumericParam("cp", lower = -1, upper = 0.1),
  makeIntegerParam("minsplit", lower = 1L, upper = 8000L),
  makeIntegerParam("minbucket", lower = 1L, upper = 4000L),
  makeIntegerParam("maxdepth", lower = 3L, upper = 20L),
  forbidden = quote(minbucket > 0.5 * minsplit)
)
# minbuket NO PUEDE ser mayor que la mitad de minsplit


# este valor debe ser 1 si se utiliza Windows
PARAM$cores <- 1

# Step 2: graba a un archivo los componentes de lista para el primer registro, escribe antes los titulos

In [7]:
loguear <- function(reg, arch = NA, folder = "./work/", ext = ".txt",
                    verbose = TRUE) {
  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  # Escribo los titulos
  if (!file.exists(archivo)) {
    linea <- paste0(
      "fecha\t",
      paste(list.names(reg), collapse = "\t"), "\n"
    )

    cat(linea, file = archivo)
  }

  # la fecha y hora
  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t",
    gsub(", ", "\t", toString(reg)), "\n"
  )

  # grabo al archivo
  cat(linea, file = archivo, append = TRUE)

  # imprimo por pantalla
  if (verbose) cat(linea)
}

- Particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa 
- particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30
- particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones


In [8]:
particionar <- function(data, division, agrupa = "", campo = "fold",
                        start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(
    function(x, y) {
      rep(y, x)
    }, division,
    seq(from = start, length.out = length(division))
  ))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}

In [9]:
# fold_test  tiene el numero de fold que voy a usar para testear,
#  entreno en el resto de los folds
# param tiene los hiperparametros del arbol

ArbolSimple <- function(fold_test, data, param) {
  # genero el modelo
  # entreno en todo MENOS el fold_test que uso para testing
  modelo <- rpart("clase_ternaria ~ .",
    data = data[fold != fold_test, ],
    xval = 0,
    control = param
  )

  # aplico el modelo a los datos de testing
  # aplico el modelo sobre los datos de testing
  # quiero que me devuelva probabilidades
  prediccion <- predict(modelo,
    data[fold == fold_test, ],
    type = "prob"
  )

  # esta es la probabilidad de baja
  prob_baja2 <- prediccion[, "BAJA+2"]

  # calculo la ganancia
  ganancia_testing <- data[fold == fold_test][
    prob_baja2 > 1 / 40,
    sum(ifelse(clase_ternaria == "BAJA+2",
      273000, -7000
    ))
  ]

  # esta es la ganancia sobre el fold de testing, NO esta normalizada
  return(ganancia_testing)
}

In [10]:
ArbolesCrossValidation <- function(data, param, qfolds, pagrupa, semilla) {
  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos
  divi <- rep(1, qfolds)

  # particiono en dataset en folds
  particionar(data, divi, seed = semilla, agrupa = pagrupa)

  ganancias <- mcmapply(ArbolSimple,
    seq(qfolds), # 1 2 3 4 5
    MoreArgs = list(data, param),
    SIMPLIFY = FALSE,
    mc.cores = PARAM$cores
  )

  data[, fold := NULL]

  ganancia_xval <- sum(unlist(ganancias))

  return(ganancia_xval)
}

In [11]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia <- function(x) {
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # param= x los hiperparametros del arbol
  # qfolds= PARAM$xval_folds  la cantidad de folds
  gans <- c()
  for (semilla in PARAM$semilla_azar)
  {
    ganancia <- ArbolesCrossValidation(dataset,
      param = x,
      qfolds = PARAM$xval_folds,
      pagrupa = "clase_ternaria",
      semilla = semilla
    )

    gans <- c(gans, ganancia)
  }
  # imprimo el vector con las ganancias de xval
  # para que se aprecia la variabilidad aun con 5-fold
  cat(gans, "\n")

  # logueo
  xx <- x
  xx$xval_repeats <- length(PARAM$semilla_azar)
  xx$xval_folds <- PARAM$xval_folds
  xx$ganancia <- mean(gans)
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch = archivo_log)

  return(mean(gans))
}

# Step 3: Aqui empieza el programa

In [12]:
# Establezco el Working Directory inicial
setwd(PARAM$home)

# cargo los datos
dataset <- fread("competencia_01.csv")
# entreno en 202103
dataset <- dataset[foto_mes == 202103]

In [13]:
# creo la carpeta donde va el experimento
#  HT  representa  Hiperparameter Tuning
dir.create("./exp/", showWarnings = FALSE)

dir.create(paste0("./exp/", PARAM$experimento, "/"),
  showWarnings = FALSE
)

In [14]:
# Establezco el Working Directory DEL EXPERIMENTO
setwd(paste0("./exp/", PARAM$experimento, "/"))

In [15]:
archivo_log <- "BO_log.txt"
archivo_BO <- "bayesiana.RDATA"

# leo si ya existe el log
#  para retomar en caso que se se corte el programa
GLOBAL_iteracion <- 0

if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
}

# Step 4:"Aqui comienza la configuracion de la Bayesian Optimization"

In [16]:
funcion_optimizar <- EstimarGanancia

In [17]:
configureMlr(show.learner.output = FALSE)

In [18]:
# configuro la busqueda bayesiana,
#  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
# minimize= FALSE estoy Maximizando la ganancia
obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar,
  minimize = FALSE,
  noisy = TRUE,
  par.set = PARAM$hs,
  has.simple.signature = FALSE
)

In [19]:
ctrl <- makeMBOControl(
  save.on.disk.at.time = 600,
  save.file.path = archivo_BO
)

In [20]:
ctrl <- setMBOControlTermination(ctrl, iters = PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())


In [21]:
surr.km <- makeLearner("regr.km",
  predict.type = "se",
  covtype = "matern3_2", control = list(trace = TRUE)
)

In [22]:
# inicio la optimizacion bayesiana
if (!file.exists(archivo_BO)) {
  run <- mbo(
    fun = obj.fun,
    learner = surr.km,
    control = ctrl
  )
} else {
  run <- mboContinue(archivo_BO)
}
# retomo en caso que ya exista

Computing y column(s) for design. Not provided.



83132000 81319000 79401000 84945000 82180000 
20230831 182058	-0.344283317326335	4205	1581	7	5	5	82195400	1
75544000 72611000 69783000 70630000 78204000 
20230831 182352	-0.64737258652458	5680	1344	9	5	5	73354400	2
66276000 65786000 66143000 66059000 66997000 
20230831 182647	-0.7357724103902	7001	2431	20	5	5	66252200	3
80325000 75670000 78904000 83979000 81361000 
20230831 182946	-0.261585744156036	5018	1824	19	5	5	80047800	4
68257000 66388000 65093000 65576000 66185000 
20230831 183312	-0.947481995684211	6994	645	14	5	5	66299800	5
69314000 69447000 64617000 68887000 69825000 
20230831 183554	-0.124807943706401	6344	156	8	5	5	68418000	6
84091000 81249000 77189000 86765000 81088000 
20230831 183916	-0.919052337198518	3003	1444	13	5	5	82076400	7
80262000 74445000 72408000 76328000 81774000 
20230831 184255	-0.251723386484664	5373	347	13	5	5	77043400	8
67137000 68516000 67788000 70245000 71421000 
20230831 184544	-0.576212055496871	6435	3129	11	5	5	69021400	9
65611000 64701000 64127000 6

[mbo] 0: cp=-0.344; minsplit=4205; minbucket=1581; maxdepth=7 : y = 8.22e+07 : 154.8 secs : initdesign

[mbo] 0: cp=-0.647; minsplit=5680; minbucket=1344; maxdepth=9 : y = 7.34e+07 : 174.2 secs : initdesign

[mbo] 0: cp=-0.736; minsplit=7001; minbucket=2431; maxdepth=20 : y = 6.63e+07 : 174.4 secs : initdesign

[mbo] 0: cp=-0.262; minsplit=5018; minbucket=1824; maxdepth=19 : y = 8e+07 : 179.2 secs : initdesign

[mbo] 0: cp=-0.947; minsplit=6994; minbucket=645; maxdepth=14 : y = 6.63e+07 : 205.7 secs : initdesign

[mbo] 0: cp=-0.125; minsplit=6344; minbucket=156; maxdepth=8 : y = 6.84e+07 : 162.8 secs : initdesign

[mbo] 0: cp=-0.919; minsplit=3003; minbucket=1444; maxdepth=13 : y = 8.21e+07 : 201.9 secs : initdesign

[mbo] 0: cp=-0.252; minsplit=5373; minbucket=347; maxdepth=13 : y = 7.7e+07 : 218.9 secs : initdesign

[mbo] 0: cp=-0.576; minsplit=6435; minbucket=3129; maxdepth=11 : y = 6.9e+07 : 168.6 secs : initdesign

[mbo] 0: cp=-0.443; minsplit=7856; minbucket=444; maxdepth=18 : y 

84980000 85302000 81277000 88004000 81963000 
20230831 190824	-0.619608285214973	2044	1020	19	5	5	84305200	17


[mbo] 1: cp=-0.62; minsplit=2044; minbucket=1020; maxdepth=19 : y = 8.43e+07 : 213.6 secs : infill_ei



...

[mbo] 298: cp=-0.535; minsplit=4413; minbucket=972; maxdepth=6 : y = 8.41e+07 : 144.4 secs : infill_ei

Saved the current state after iteration 299 in the file bayesiana.RDATA.



66276000 65065000 66080000 66059000 66997000 
20230901 112632	-0.141416481814274	7216	2817	10	5	5	66095400	315


[mbo] 299: cp=-0.141; minsplit=7216; minbucket=2817; maxdepth=10 : y = 6.61e+07 : 160.2 secs : infill_ei



64365000 61005000 63686000 66787000 64526000 
20230901 113039	-0.124928741438601	8000	2	14	5	5	64073800	316


[mbo] 300: cp=-0.125; minsplit=8000; minbucket=2; maxdepth=14 : y = 6.41e+07 : 243.7 secs : infill_ei

Saved the final state in the file bayesiana.RDATA

